# Deep Fashion 2 visualization

The main purpose of this notebook is to understand the contents of the Deep Fashion dataset and have answers to questions like:

- How many images per category are there?
- How many clothing objects per image are there?
- Is the quality of the image good enough?
- What does the attributes tell us about the images?

In [ ]:
from typing import List

import skimage.io

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from masks import get_mask
from data_loader import load_training_df
from visualization import display_instances

In [ ]:
dataset_path = 'dataset'

## Read dataset

Read images and annotations in training.

In [ ]:
df = load_training_df(dataset_path)

Map ordinal categories (e.g. scale, occlusion and viewpoint) into categorical columns for better understanding (values according to documentation).

In [ ]:
df['scale_categorical'] = df['scale'].map(
    {1: 'small_scale', 2: 'modest_scale', 3: 'large_scale'}
)
df['zoom_in_categorical'] = df['zoom_in'].map(
    {1: 'no_zoom_in', 2: 'medium_zoom_in', 3: 'large_zoom_in'}
)
df['viewpoint_categorical'] = df['viewpoint'].map(
    {1: 'no_wear', 2: 'frontal_viewpoint', 3: 'side_or_back_viewpoint'}
)
df['occlusion_categorical'] = df['occlusion'].map(
    {1: 'slight_occlusion', 2: 'medium_occlusion', 3: 'heavy_occlusion'}
)

Let's visualize some examples.

In [ ]:
df.sample(3)

## Data visualization

Let's observe random examples for the different features.

### Display image by category id

In [ ]:
def _samples_per_category(df: pd.DataFrame,
                          column: str,
                          n_samples: int = 7) -> pd.DataFrame:
    return df.groupby(column)\
             .apply(lambda x: x.sample(n_samples))\
             .reset_index(level=0, drop=True)\
             .reset_index()
    

column = 'category_name'
display_instances(_samples_per_category(df, column, n_samples=2),
                  title_column=column,
                  n_cols=9)

In [ ]:
df[column].value_counts().plot.bar(figsize=(20, 4), rot=45)

We see some categories are highly close to one another (e.g. sling dress and short sleeve dress).

### Display images by source information

In [ ]:
column = 'source'
display_instances(_samples_per_category(df, column, n_samples=8),
                  title_column=column,
                  n_cols=8)

In [ ]:
df[column].value_counts().plot.bar(figsize=(10, 4), rot=45)

We see the `shop` images have much higher quality than `user` images. We see that there are ~3 times more shop images than user images.

### Display image by viewpoint information

In [ ]:
column = 'viewpoint_categorical'
display_instances(_samples_per_category(df, column, n_samples=4),
                  title_column=column,
                  n_cols=6)

In [ ]:
df[column].value_counts().plot.bar(figsize=(10, 4), rot=45)

We see that the viewpoint information can be ambiguous as side viewpoint (which most of the time are mostly frontal vies) and back viewpoint are tagged in the same category. Moreover, we observe that most of the images fall in frontal category.

### Display image by scale information

In [ ]:
column = 'scale_categorical'
display_instances(_samples_per_category(df, column, n_samples=4),
                  title_column=column,
                  n_cols=6)

In [ ]:
df[column].value_counts().plot.bar(figsize=(20, 4), rot=45)

Again, we see that scale is not very informative, as `small` and `modest` scale refer to very similar kind of images. However, they seem to properly tag those which are zoomed in pictures.

### Display image by zoom-in information

In [ ]:
column = 'zoom_in_categorical'
display_instances(_samples_per_category(df, column, n_samples=4),
                  title_column=column,
                  n_cols=6)

In [ ]:
df[column].value_counts().plot.bar(figsize=(20, 4), rot=45)

We observe that this feature gives little information about the content, as we see that similar images appear in different categories such as `no_zoom_in` and `large_zoom_in`.

### Display image by occlusion information

In [ ]:
column = 'occlusion_categorical'
display_instances(_samples_per_category(df, column, n_samples=4),
                  title_column=column,
                  n_cols=6)

In [ ]:
df[column].value_counts().plot.bar(figsize=(20, 4), rot=45)

Again, it is not clear what is the criteria used to tag image occlusion.

### Display image pairs

In [ ]:
pair_ids = list(df.sample(1)['pair_id'].values)
sample_pairs_df = df[df['pair_id'].isin(pair_ids)].drop_duplicates(['image_path'])
display_instances(sample_pairs_df, title_column='source', n_cols=9)

Shop and user image does not need to be from the same size or color. That can be read in the `style` field (see [documentation](https://github.com/switchablenorms/DeepFashion2)).

## Clothing elements per image: stats

In [ ]:
clothes_per_image = df.groupby('image_path')['category_id'].count()
mean, std = clothes_per_image.mean(), clothes_per_image.std()
print(f'Clothes per image: {mean:.2f} +- {std:.2f}')

## Mask generation

Compute mask image from examples.

In [ ]:
def _display_masks(image: np.ndarray, masks: List, ax=None):
    # Display image
    if ax is not None:
        axis = ax
    else:
        plt.figure(figsize=(8, 15))
        axis = plt
    axis.imshow(image)
    
    # Display all masks
    for mask in masks:
        axis.imshow(mask, alpha=0.25, vmin=-1.0, vmax=1.0)

        
def instance_to_mask(row: pd.Series) -> np.ndarray:
    image = skimage.io.imread(row['image_path'])
    image_height, image_width = image.shape[:2]
    return get_mask(image_height,
                    image_width,
                    polygons=row['segmentation'],
                    category_id=int(row['category_id']))


def display_instance_mask(row: pd.Series, ax) -> None:
    masks = [instance_to_mask(row)]
    image = skimage.io.imread(row['image_path'])
    _display_masks(image, masks, ax=ax)
    ax.set_title(row["category_name"])
    ax.axis('off')
    
samples = df.sample(12)
display_instances(samples, display_fn=display_instance_mask, n_cols=6)

We see that, in many cases, polygons defining the clothing area are quite sharp and do not properly wrap the clothes margin.

Let's now visualize examples of images with all masks in it.

In [ ]:
def display_all_instance_masks(row: pd.Series, ax) -> None:
    items = df[df.index == row.name]
    masks = items.apply(instance_to_mask, axis=1).values.tolist()
    image = skimage.io.imread(row['image_path'])
    _display_masks(image, masks, ax=ax)
    # Displau call categories
    categories = items["category_name"].values.tolist()
    ax.set_title(f'{categories}', fontsize=8)
    ax.axis('off')

samples = df.sample(12)
display_instances(samples, display_fn=display_all_instance_masks, n_cols=6)